#### Load in Movies Data

In [1]:
import pandas as pd
import numpy as np
import re

movies = pd.read_csv("omdb_enriched_data.csv")
movies.head(1)

,title,release_date,original_language,genres,budget,revenue,runtime,year,Rated,Poster,Ratings,Metascore,imdbRating,imdbVotes,imdbID
0,Meg 2: The Trench,2023-08-02,en,Action-Science Fiction-Horror,129000000.0,352056482.0,116.0,2023.0,PG-13,posters/Meg_2:_The_Trench_2023_photo.jpg,"[{'Source': 'Internet Movie Database', 'Value'...",NaN,5.0,"86,089",tt9224104


##### Produce `month` column, drop `release_date`, select only top `genre`

In [2]:
# create 'month' column for later join with crime data
movies['release_date'] = pd.to_datetime(movies['release_date'])
movies['Month'] = movies['release_date'].dt.strftime('%m-%Y')
movies = movies.drop(columns=['release_date'])

# Only use top genre
movies['genres'] = movies['genres'].str.split('-').str[0]

movies.head(2)

,title,original_language,genres,budget,revenue,runtime,year,Rated,Poster,Ratings,Metascore,imdbRating,imdbVotes,imdbID,Month
0,Meg 2: The Trench,en,Action,129000000.0,352056482.0,116.0,2023.0,PG-13,posters/Meg_2:_The_Trench_2023_photo.jpg,"[{'Source': 'Internet Movie Database', 'Value'...",NaN,5.0,"86,089",tt9224104,08-2023
1,The Pope's Exorcist,en,Horror,18000000.0,65675816.0,103.0,2023.0,R,posters/The_Pope's_Exorcist_2023_photo.jpg,"[{'Source': 'Internet Movie Database', 'Value'...",45.0,6.1,"88,285",tt13375076,04-2023


##### Inspect Ratings field

In [3]:
for i in range(1,5):
    print(movies.iloc[i].Ratings)

[{'Source': 'Internet Movie Database', 'Value': '6.1/10'}, {'Source': 'Rotten Tomatoes', 'Value': '50%'}, {'Source': 'Metacritic', 'Value': '45/100'}]
[{'Source': 'Internet Movie Database', 'Value': '7.9/10'}, {'Source': 'Rotten Tomatoes', 'Value': '78%'}]
[{'Source': 'Internet Movie Database', 'Value': '6.0/10'}, {'Source': 'Rotten Tomatoes', 'Value': '51%'}, {'Source': 'Metacritic', 'Value': '42/100'}]
[{'Source': 'Internet Movie Database', 'Value': '8.5/10'}, {'Source': 'Rotten Tomatoes', 'Value': '92%'}]


##### Extract Ratings
We have IMDb, Rotten Tomatoes, and Metacritic ratings.  
Extract and convert to equal units

In [4]:
# Function to extract and convert ratings based on the source
def extract_rating(ratings_list, source_name):
    if isinstance(ratings_list, list):
        for rating in ratings_list:
            if rating['Source'] == source_name:
                value = rating['Value']
                # Process each source differently
                if source_name == 'Internet Movie Database':
                    return float(value.split('/')[0])  # Scale is already 0-10
                elif source_name == 'Rotten Tomatoes':
                    return float(value.replace('%', '')) / 10  # Convert % to 0-10
                elif source_name == 'Metacritic':
                    return float(value.split('/')[0]) / 10  # Convert 0-100 scale to 0-10
    return np.nan


##### Creating RT_Rating
Based on inspection the IMDb Ratings match the values in the imdbRating column. The Metacritic ratings are very sparse so we will leave those out for now.  So, just going to extract the RT ratings

In [5]:
# Ensure 'Ratings' column is a list of dictionaries
movies['Ratings'] = movies['Ratings'].apply(lambda x: eval(x) if isinstance(x, str) else x)

# Extract only the Rotten Tomatoes rating and create a new column
movies['RT_Rating'] = movies['Ratings'].apply(lambda x: extract_rating(x, 'Rotten Tomatoes'))

# Drop the original 'Ratings' column if it's no longer needed
movies = movies.drop(columns=['Ratings', 'Metascore'])

movies.head(2)

,title,original_language,genres,budget,revenue,runtime,year,Rated,Poster,imdbRating,imdbVotes,imdbID,Month,RT_Rating
0,Meg 2: The Trench,en,Action,129000000.0,352056482.0,116.0,2023.0,PG-13,posters/Meg_2:_The_Trench_2023_photo.jpg,5.0,"86,089",tt9224104,08-2023,2.7
1,The Pope's Exorcist,en,Horror,18000000.0,65675816.0,103.0,2023.0,R,posters/The_Pope's_Exorcist_2023_photo.jpg,6.1,"88,285",tt13375076,04-2023,5.0


#### Process images
Convert to RGB, Resize, Convert to array, Normalize Pixel values

In [6]:
from PIL import Image

# function to process images
def preprocess_poster(poster, target_size=(224, 224)):
    if pd.isna(poster):  # Check if the poster path is NaN
        return None
    try:
        # Open image
        with Image.open(poster) as img:
            # Convert to RGB (ResNet expects 3 channels)
            img = img.convert("RGB")
            # Resize to the ResNet input size
            img = img.resize(target_size, Image.LANCZOS)
            # Convert image to NumPy array
            img_array = np.array(img)
            # Normalize pixel values to [0, 1] range
            img_array = img_array / 255.0
            return img_array
    except Exception as e:
        print(f"Error loading poster at {poster}: {e}")
        return None  # Return None if there was an error


In [7]:
# Apply the function to preprocess images, creating `Poster_Data`
movies['Poster_Data'] = movies['Poster'].apply(lambda path: preprocess_poster(path))

# Check shape
print(movies.shape)

# Drop rows where Poster_Data is None (if necessary)
movies = movies.dropna(subset=['Poster_Data'])

print(movies.shape)
movies.head(2).Poster_Data

Error loading poster at posters/Meg_2:_The_Trench_2023_photo.jpg: [Errno 2] No such file or directory: 'posters/Meg_2:_The_Trench_2023_photo.jpg'
Error loading poster at posters/Transformers:_Rise_of_the_Beasts_2023_photo.jpg: [Errno 2] No such file or directory: 'posters/Transformers:_Rise_of_the_Beasts_2023_photo.jpg'
Error loading poster at posters/Dune:_Part_Two_2024_photo.jpg: [Errno 2] No such file or directory: 'posters/Dune:_Part_Two_2024_photo.jpg'
Error loading poster at posters/Ant-Man_and_the_Wasp:_Quantumania_2023_photo.jpg: [Errno 2] No such file or directory: 'posters/Ant-Man_and_the_Wasp:_Quantumania_2023_photo.jpg'
Error loading poster at posters/Insidious:_The_Red_Door_2023_photo.jpg: [Errno 2] No such file or directory: 'posters/Insidious:_The_Red_Door_2023_photo.jpg'
Error loading poster at posters/Spider-Man:_Across_the_Spider-Verse_2023_photo.jpg: [Errno 2] No such file or directory: 'posters/Spider-Man:_Across_the_Spider-Verse_2023_photo.jpg'
Error loading poster

Error loading poster at posters/Pirates_of_the_Caribbean:_The_Curse_of_the_Black_Pearl_2003_photo.jpg: [Errno 2] No such file or directory: 'posters/Pirates_of_the_Caribbean:_The_Curse_of_the_Black_Pearl_2003_photo.jpg'
Error loading poster at posters/xXx:_Return_of_Xander_Cage_2017_photo.jpg: [Errno 2] No such file or directory: 'posters/xXx:_Return_of_Xander_Cage_2017_photo.jpg'
Error loading poster at posters/The_Jack_in_the_Box:_Awakening_2022_photo.jpg: [Errno 2] No such file or directory: 'posters/The_Jack_in_the_Box:_Awakening_2022_photo.jpg'
Error loading poster at posters/Ghostbusters:_Afterlife_2021_photo.jpg: [Errno 2] No such file or directory: 'posters/Ghostbusters:_Afterlife_2021_photo.jpg'
Error loading poster at posters/Spider-Man:_Homecoming_2017_photo.jpg: [Errno 2] No such file or directory: 'posters/Spider-Man:_Homecoming_2017_photo.jpg'
Error loading poster at posters/Harry_Potter_and_the_Deathly_Hallows:_Part_1_2010_photo.jpg: [Errno 2] No such file or directory: 

Error loading poster at posters/Spirit:_Stallion_of_the_Cimarron_2002_photo.jpg: [Errno 2] No such file or directory: 'posters/Spirit:_Stallion_of_the_Cimarron_2002_photo.jpg'
Error loading poster at posters/PAW_Patrol:_The_Movie_2021_photo.jpg: [Errno 2] No such file or directory: 'posters/PAW_Patrol:_The_Movie_2021_photo.jpg'
Error loading poster at posters/Madagascar_3:_Europe's_Most_Wanted_2012_photo.jpg: [Errno 2] No such file or directory: "posters/Madagascar_3:_Europe's_Most_Wanted_2012_photo.jpg"
Error loading poster at posters/Are_You_There_God?_It's_Me,_Margaret._2023_photo.jpg: [Errno 22] Invalid argument: "posters/Are_You_There_God?_It's_Me,_Margaret._2023_photo.jpg"
Error loading poster at posters/The_Hobbit:_An_Unexpected_Journey_2012_photo.jpg: [Errno 2] No such file or directory: 'posters/The_Hobbit:_An_Unexpected_Journey_2012_photo.jpg'
Error loading poster at posters/Crank:_High_Voltage_2009_photo.jpg: [Errno 2] No such file or directory: 'posters/Crank:_High_Voltage_

Error loading poster at posters/The_Purge:_Election_Year_2016_photo.jpg: [Errno 2] No such file or directory: 'posters/The_Purge:_Election_Year_2016_photo.jpg'
Error loading poster at posters/The_Mummy:_Tomb_of_the_Dragon_Emperor_2008_photo.jpg: [Errno 2] No such file or directory: 'posters/The_Mummy:_Tomb_of_the_Dragon_Emperor_2008_photo.jpg'
Error loading poster at posters/Aliens_vs_Predator:_Requiem_2007_photo.jpg: [Errno 2] No such file or directory: 'posters/Aliens_vs_Predator:_Requiem_2007_photo.jpg'
Error loading poster at posters/Perfume:_The_Story_of_a_Murderer_2006_photo.jpg: [Errno 2] No such file or directory: 'posters/Perfume:_The_Story_of_a_Murderer_2006_photo.jpg'
Error loading poster at posters/G.I._Joe:_The_Rise_of_Cobra_2009_photo.jpg: [Errno 2] No such file or directory: 'posters/G.I._Joe:_The_Rise_of_Cobra_2009_photo.jpg'
Error loading poster at posters/X-Men:_Apocalypse_2016_photo.jpg: [Errno 2] No such file or directory: 'posters/X-Men:_Apocalypse_2016_photo.jpg'


Error loading poster at posters/Escape_Room:_Tournament_of_Champions_2021_photo.jpg: [Errno 2] No such file or directory: 'posters/Escape_Room:_Tournament_of_Champions_2021_photo.jpg'
Error loading poster at posters/Resident_Evil:_Damnation_2012_photo.jpg: [Errno 2] No such file or directory: 'posters/Resident_Evil:_Damnation_2012_photo.jpg'
Error loading poster at posters/Zathura:_A_Space_Adventure_2005_photo.jpg: [Errno 2] No such file or directory: 'posters/Zathura:_A_Space_Adventure_2005_photo.jpg'
Error loading poster at posters/Babe:_Pig_in_the_City_1998_photo.jpg: [Errno 2] No such file or directory: 'posters/Babe:_Pig_in_the_City_1998_photo.jpg'
Error loading poster at posters/Underworld:_Rise_of_the_Lycans_2009_photo.jpg: [Errno 2] No such file or directory: 'posters/Underworld:_Rise_of_the_Lycans_2009_photo.jpg'
Error loading poster at posters/Kung_Pow:_Enter_the_Fist_2002_photo.jpg: [Errno 2] No such file or directory: 'posters/Kung_Pow:_Enter_the_Fist_2002_photo.jpg'
Error 

...

Error loading poster at posters/Rise:_Blood_Hunter_2007_photo.jpg: [Errno 2] No such file or directory: 'posters/Rise:_Blood_Hunter_2007_photo.jpg'
Error loading poster at posters/Horrid_Henry:_The_Movie_2011_photo.jpg: [Errno 2] No such file or directory: 'posters/Horrid_Henry:_The_Movie_2011_photo.jpg'
Error loading poster at posters/Fur:_An_Imaginary_Portrait_of_Diane_Arbus_2006_photo.jpg: [Errno 2] No such file or directory: 'posters/Fur:_An_Imaginary_Portrait_of_Diane_Arbus_2006_photo.jpg'
Error loading poster at posters/Kevin_Hart:_What_Now?_2016_photo.jpg: [Errno 2] No such file or directory: 'posters/Kevin_Hart:_What_Now?_2016_photo.jpg'
Error loading poster at posters/Red_Dog:_True_Blue_2016_photo.jpg: [Errno 2] No such file or directory: 'posters/Red_Dog:_True_Blue_2016_photo.jpg'
Error loading poster at posters/Ninja_III:_The_Domination_1984_photo.jpg: [Errno 2] No such file or directory: 'posters/Ninja_III:_The_Domination_1984_photo.jpg'
Error loading poster at posters/What

Error loading poster at posters/Fast_Track:_No_Limits_2008_photo.jpg: [Errno 2] No such file or directory: 'posters/Fast_Track:_No_Limits_2008_photo.jpg'
Error loading poster at posters/Jimi:_All_Is_by_My_Side_2013_photo.jpg: [Errno 2] No such file or directory: 'posters/Jimi:_All_Is_by_My_Side_2013_photo.jpg'
Error loading poster at posters/Porn_Star:_The_Legend_of_Ron_Jeremy_2001_photo.jpg: [Errno 2] No such file or directory: 'posters/Porn_Star:_The_Legend_of_Ron_Jeremy_2001_photo.jpg'
Error loading poster at posters/2016:_Obama's_America_2012_photo.jpg: [Errno 2] No such file or directory: "posters/2016:_Obama's_America_2012_photo.jpg"
Error loading poster at posters/Bat★21_1988_photo.jpg: [Errno 2] No such file or directory: 'posters/Bat★21_1988_photo.jpg'
Error loading poster at posters/(Untitled)_2009_photo.jpg: [Errno 2] No such file or directory: 'posters/(Untitled)_2009_photo.jpg'
Error loading poster at posters/Fireman_Sam:_Set_for_Action!_2018_photo.jpg: [Errno 2] No such f

Error loading poster at posters/The_Lost_World:_Jurassic_Park_1997_photo.jpg: [Errno 2] No such file or directory: 'posters/The_Lost_World:_Jurassic_Park_1997_photo.jpg'
Error loading poster at posters/The_Wedding_Party_2:_Destination_Dubai_2017_photo.jpg: [Errno 2] No such file or directory: 'posters/The_Wedding_Party_2:_Destination_Dubai_2017_photo.jpg'
Error loading poster at posters/Kanye_West:_DONDA_Experience_Performance_2_22_22_2022_photo.jpg: [Errno 2] No such file or directory: 'posters/Kanye_West:_DONDA_Experience_Performance_2_22_22_2022_photo.jpg'
Error loading poster at posters/They_Call_Me_Bruce?_1982_photo.jpg: [Errno 22] Invalid argument: 'posters/They_Call_Me_Bruce?_1982_photo.jpg'
Error loading poster at posters/UFC_269:_Oliveira_vs._Poirier_2021_photo.jpg: [Errno 2] No such file or directory: 'posters/UFC_269:_Oliveira_vs._Poirier_2021_photo.jpg'
Error loading poster at posters/UFC_95:_Sanchez_vs_Stevenson_2009_photo.jpg: [Errno 2] No such file or directory: 'posters

Error loading poster at posters/Do_You_Want_To_Win?_2017_photo.jpg: [Errno 22] Invalid argument: 'posters/Do_You_Want_To_Win?_2017_photo.jpg'
Error loading poster at posters/The_Gamers:_Hands_of_Fate_2013_photo.jpg: [Errno 2] No such file or directory: 'posters/The_Gamers:_Hands_of_Fate_2013_photo.jpg'
Error loading poster at posters/Resident_Evil:_Afterlife_2010_photo.jpg: [Errno 2] No such file or directory: 'posters/Resident_Evil:_Afterlife_2010_photo.jpg'
Error loading poster at posters/Hillsong:_Let_Hope_Rise_2016_photo.jpg: [Errno 2] No such file or directory: 'posters/Hillsong:_Let_Hope_Rise_2016_photo.jpg'
Error loading poster at posters/Bad_Ben:_The_Mandela_Effect_2018_photo.jpg: [Errno 2] No such file or directory: 'posters/Bad_Ben:_The_Mandela_Effect_2018_photo.jpg'
Error loading poster at posters/Peter_Gabriel:_New_Blood,_Live_In_London_2011_photo.jpg: [Errno 2] No such file or directory: 'posters/Peter_Gabriel:_New_Blood,_Live_In_London_2011_photo.jpg'
Error loading poster

1    [[[0.08235294117647059, 0.00784313725490196, 0...
2    [[[0.07450980392156863, 0.07450980392156863, 0...
Name: Poster_Data, dtype: object

#### Load in crime data

In [8]:
Reports = pd.read_csv('States_Reports.csv')
Clearances = pd.read_csv('States_Clearances.csv')
Reports.head()

,Month,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,District of Columbia,...,South Dakota,Tennessee,Texas,Utah,Vermont,Utah.1,Washington,West Virginia,Wisconsin,Wyoming
0,10-2014,8196.0,1094.0,7581.0,4719.0,34958.0,4151.0,3227.0,1652.0,2270.0,...,867.0,12574.0,35436.0,2655.0,260.0,2655.0,6685.0,1376.0,3627.0,533.0
1,11-2014,7729.0,1123.0,6942.0,4060.0,31782.0,3842.0,2838.0,1455.0,1857.0,...,792.0,10996.0,31637.0,2475.0,215.0,2475.0,6279.0,1256.0,3266.0,560.0
2,12-2014,7412.0,1058.0,6971.0,4037.0,32120.0,3682.0,2763.0,1461.0,1597.0,...,774.0,11483.0,32388.0,2430.0,167.0,2430.0,6340.0,1224.0,3377.0,499.0
3,01-2015,8026.0,1240.0,7383.0,4624.0,33805.0,4173.0,2629.0,1521.0,1877.0,...,885.0,11719.0,31617.0,2655.0,231.0,2655.0,6590.0,1295.0,3374.0,511.0
4,02-2015,6669.0,977.0,6851.0,3791.0,31436.0,3578.0,2311.0,1193.0,1501.0,...,743.0,9423.0,28801.0,2437.0,240.0,2437.0,6102.0,966.0,2815.0,474.0


##### Filter movies for those with relevant crime data available

In [9]:
# Only use movies where we have crime data 
movies = movies[pd.to_datetime(movies['Month'])>= pd.to_datetime('2014-10-01')]  # no data before Octiber 2014
movies = movies[pd.to_datetime(movies['Month'])<= pd.to_datetime('2023-12-31')]  # or after 2023
movies = movies[movies['Poster_Data'].notna()]

Reports = pd.merge(movies[['title', 'Month']], Reports, on='Month', how='inner')
Reports.shape
Reports.head()

C:\Users\sfure\AppData\Local\Temp\ipykernel_20620\1576980597.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  movies = movies[pd.to_datetime(movies['Month'])>= pd.to_datetime('2014-10-01')]  # no data before Octiber 2014
C:\Users\sfure\AppData\Local\Temp\ipykernel_20620\1576980597.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  movies = movies[pd.to_datetime(movies['Month'])<= pd.to_datetime('2023-12-31')]  # or after 2023


,title,Month,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,...,South Dakota,Tennessee,Texas,Utah,Vermont,Utah.1,Washington,West Virginia,Wisconsin,Wyoming
0,The Pope's Exorcist,04-2023,6489.0,1260.0,8431.0,5409.0,39549.0,5973.0,2359.0,1361.0,...,1044.0,12802.0,41292.0,3184.0,343.0,3184.0,7236.0,1446.0,4272.0,584.0
1,Knights of the Zodiac,04-2023,6489.0,1260.0,8431.0,5409.0,39549.0,5973.0,2359.0,1361.0,...,1044.0,12802.0,41292.0,3184.0,343.0,3184.0,7236.0,1446.0,4272.0,584.0
2,Guy Ritchie's The Covenant,04-2023,6489.0,1260.0,8431.0,5409.0,39549.0,5973.0,2359.0,1361.0,...,1044.0,12802.0,41292.0,3184.0,343.0,3184.0,7236.0,1446.0,4272.0,584.0
3,Evil Dead Rise,04-2023,6489.0,1260.0,8431.0,5409.0,39549.0,5973.0,2359.0,1361.0,...,1044.0,12802.0,41292.0,3184.0,343.0,3184.0,7236.0,1446.0,4272.0,584.0
4,The Super Mario Bros. Movie,04-2023,6489.0,1260.0,8431.0,5409.0,39549.0,5973.0,2359.0,1361.0,...,1044.0,12802.0,41292.0,3184.0,343.0,3184.0,7236.0,1446.0,4272.0,584.0


#### Produce feature vector from `Poster_Data` using ResNet50  
We do not want to predict with ResNet. We want to acquire features that we can combine with other movie details to then make our predictions.  
So we will remove the final classification layer and save the resulting features as `Poster_Features`

In [10]:
import tensorflow as tf

# Convert Poster_Data to a 4D tensor (batch of images) for ResNet
poster_images = np.stack(movies['Poster_Data'].values)

# Load a pre-trained ResNet model in TensorFlow
# 'include_top=False' removes the final classification layers, and 'pooling='avg'' is needed to get 1D vector
resnet = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, pooling='avg')

# Apply ResNet to images
poster_features = resnet.predict(poster_images)

# Store the extracted features back into the DataFrame
movies['Poster_Features'] = list(poster_features)


48/48 ━━━━━━━━━━━━━━━━━━━━ 66s 1s/step


#### Isolate and Scale Variables

In [11]:
from sklearn.preprocessing import StandardScaler

X = movies[['runtime', 'revenue']].values
scaler = StandardScaler()
X = scaler.fit_transform(X)
y = Reports['Alabama'].values

##### Create some additional metrics

In [12]:
# Define R squared
def r2(y_true, y_pred):
    ss_res = tf.reduce_sum(tf.square(y_true - y_pred))
    ss_tot = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true)))
    return 1 - (ss_res / ss_tot)

# Define root mean squared error
def rmse(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_pred - y_true)))

#### Build and compile combined neural net
Will take in both selected movie details and generated `Poster_Features`

In [13]:
from tensorflow.keras.layers import Dense, Flatten, Concatenate, Input
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model

image_input = Input(shape=(poster_features.shape[1],))
num_input = Input(shape=(X.shape[1],))

dense_num = Dense(64, activation='relu')(num_input)
dense_num = Dense(32, activation='relu')(dense_num)

merged = Concatenate()([image_input, dense_num])

combined_dense = Dense(128, activation='relu')(merged)
combined_dense = Dense(64, activation='relu')(combined_dense)
output = Dense(1, activation='linear')(combined_dense)

model = Model(inputs=[image_input, num_input], outputs=output)
model.compile(optimizer='adam', loss='mae', metrics=['mse', r2, rmse])

#### Fit the model

In [14]:
history = model.fit(
    [poster_features, X],
    y,
    epochs=10,
    batch_size=64)

Epoch 1/10


C:\Users\sfure\anaconda3\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_176', 'keras_tensor_177']. Received: the structure of inputs=('*', '*')
  warnings.warn(


24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 1274.7388 - mse: 1675606.3750 - r2: -2293.4160 - rmse: 1294.1858
Epoch 2/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1156.1504 - mse: 1396261.2500 - r2: -1816.1266 - rmse: 1180.6638
Epoch 3/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 830.4680 - mse: 756935.9375 - r2: -1104.9828 - rmse: 861.0941
Epoch 4/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 257.8418 - mse: 103919.9219 - r2: -124.8381 - rmse: 318.5766
Epoch 5/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 180.3170 - mse: 64057.4883 - r2: -78.7394 - rmse: 251.5376
Epoch 6/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 174.7085 - mse: 64619.2773 - r2: -74.0183 - rmse: 252.5008
Epoch 7/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 160.7906 - mse: 59345.0977 - r2: -71.3830 - rmse: 246.4400
Epoch 8/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 161.3073 - mse: 57204.6719 - r2: -71.0701 - rmse: 243.5572
Epoch 9/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 

## 